# Homework 3

## In-class exercise 1

### Step 1

Load the request per capita dataset from https://storage.googleapis.com/python-public-policy/data/311_community_districts.csv.zip as `requests_by_cd` and display it.

In [ ]:
# your code here

### Step 2

Make a [histogram](https://plotly.com/python/histograms/) of the requests per capita.

In [ ]:
# your code here

## In-class exercise 2

Take the scatterplot example from [the lecture](lecture_3.ipynb) and [add a trendline](https://plotly.com/python/linear-fits/).

In [ ]:
# your code here

## Coding

We are going to look at the population count of different community districts over time.

In [ ]:
import plotly.express as px

# boilerplate for allowing PDF export
import plotly.io as pio

pio.renderers.default = "notebook_connected+pdf"

### Step 1

Read the data from the [New York City Population By Community Districts](https://data.cityofnewyork.us/City-Government/New-York-City-Population-By-Community-Districts/xi7c-iiu2/data) data set into a DataFrame called `pop_by_cd`. To get the URL:

1. Visit the page linked above.
1. Click `Export`.
1. Right-click `CSV`.
1. Click `Copy Link Address` (or `Location`, depending on your browser).

In [ ]:
# your code here

### Step 2

Prepare the data. Use the following code to [reshape](https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html#reshaping-by-melt) the DataFrame to have one row per community district per Census year.

In [ ]:
# turn the population columns into rows
populations = pd.melt(
    pop_by_cd,
    id_vars=["Borough", "CD Number", "CD Name"],
    var_name="year",
    value_name="population",
)

# turn the years into numbers
populations.year = populations.year.str.replace(" Population", "").astype(int)

populations

### Step 3

Create a line chart of the population over time for each community district in Manhattan. There should be one line for each.

See the Plotly [Line Plot with column encoding color](https://plotly.com/python/line-charts/#line-plots-with-column-encoding-color) examples.

In [ ]:
# your code here

### Step 4

We are going to do some mapping using the `pop_by_cd` DataFrame from before. To do so, we need `borocd`s. Create that column with the values filled in. (See [Lecture 2](lecture_2.ipynb).)

In [ ]:
# your code here

### Step 5

Let make a [choropleth map](https://www.data-to-viz.com/graph/choropleth.html) showing the population change from 2000 to 2010 for each community district. Adapt the `.choropleth_mapbox()` example in [Lecture 3](lecture_3.ipynb).

In [ ]:
# your code here

### Step 6

***Analysis: Washington Heights and Inwood (the tall skinny community district at the top of Manhattan) are "up and coming" neighborhoods. In a few sentences: Why might might the population have decreased?***

YOUR ANSWER HERE

Then, read the first three paragraphs of the `Demographics` section of [An Economic Snapshot of Washington Heights and Inwood from June 2015](https://www.osc.state.ny.us/osdc/rpt2-2016.pdf#page=2).

Now [turn in the assignment](https://python-public-policy.afeld.me/en/latest/README.html#turning-in-assignments).

## Tutorials

1. Go through the first third of [Time Series Analysis with Pandas](https://www.dataquest.io/blog/tutorial-time-series-analysis-with-pandas/), up until the "Visualizing time series data" section.
1. Read [how to handle time series data in pandas](https://pandas.pydata.org/pandas-docs/stable/getting_started/intro_tutorials/09_timeseries.html).
1. Read the [Data Design Standards](https://xdgov.github.io/data-design-standards/).
1. Watch [this talk on audification/sonification](https://www.youtube.com/watch?v=55dIfA7C038). We won't be doing so in this class, but hopefully will provide some inspiration about different ways that data can be represented.
1. Optional: Read about other tools and techniques for visualization in Python.
   - [PyViz](https://pyviz.org/)
   - [Python Graph Gallery](https://www.python-graph-gallery.com/)

## Participation

Reminder about the [between-class participation requirement](https://python-public-policy.afeld.me/en/latest/syllabus.html#participation).